In [1]:
import pandas as pd
from itertools import permutations
import numpy as np
import random

In [28]:
distance_matrix = [
    [0, 10, 15, 20],
    [10, 0, 35, 25],
    [15, 35, 0, 30],
    [20, 25, 30, 0]
]

distance_matrix_bis = 100 * np.random.rand(100,100)

In [43]:
def exact_method(distance_matrix, circular=True):
    n = len(distance_matrix)
    cities = range(n)
    min_cost = float('inf')
    best_route = None
    for perm in permutations(cities):
        perm = list(perm)
        cost = sum(distance_matrix[perm[i]][perm[i + 1]] for i in range(n - 1))
        if circular:
            cost += distance_matrix[perm[-1]][perm[0]]
            
            perm.append(perm[0])
        if cost < min_cost:
            min_cost = cost
            best_route = perm
    return min_cost, best_route

print(exact_method(distance_matrix))



(80, [0, 1, 3, 2, 0])


In [48]:

def Glouton(distance_matrix,circular =True):
    n = len(distance_matrix)
    unvisited = set(range(n))
    current_city = 0
    tour = [current_city]
    unvisited.remove(current_city)
    while unvisited:
        next_city = min(unvisited, key=lambda city: distance_matrix[current_city][city])
        tour.append(next_city)
        unvisited.remove(next_city)
        current_city = next_city
    if circular:
        tour.append(tour[0])
    cost = sum(distance_matrix[tour[i]][tour[i + 1]] for i in range(len(tour) - 1))
    return cost, tour



print(Glouton(distance_matrix_bis))


(478.72550666795485, [0, 72, 49, 5, 16, 37, 87, 27, 54, 59, 79, 52, 43, 78, 36, 24, 53, 97, 46, 18, 1, 89, 48, 91, 81, 94, 85, 64, 40, 11, 23, 39, 88, 55, 58, 21, 7, 57, 95, 29, 66, 32, 82, 38, 3, 17, 61, 15, 90, 42, 65, 75, 34, 47, 84, 12, 63, 50, 13, 31, 93, 67, 70, 41, 35, 69, 22, 14, 9, 8, 2, 68, 6, 76, 25, 71, 28, 56, 26, 74, 4, 62, 86, 51, 73, 10, 33, 45, 96, 98, 19, 30, 20, 99, 77, 80, 44, 92, 83, 60, 0])


In [31]:
def simulated_annealing(distance_matrix, initial_temp=1000, cooling_rate=0.995, max_iter=1000, circular=True):
    n = len(distance_matrix)
    current_solution = list(range(n))
    random.shuffle(current_solution)
    if circular:
        current_cost = sum(distance_matrix[current_solution[i]][current_solution[(i + 1) % n]] for i in range(n))
    else:
        current_cost = sum(distance_matrix[current_solution[i]][current_solution[i + 1]] for i in range(n - 1))
    best_solution = current_solution[:]
    best_cost = current_cost
    temp = initial_temp
    for _ in range(max_iter):
        i, j = random.sample(range(n), 2)
        new_solution = current_solution[:]
        new_solution[i], new_solution[j] = new_solution[j], new_solution[i]
        if circular:
            new_cost = sum(distance_matrix[new_solution[i]][new_solution[(i + 1) % n]] for i in range(n))
        else:
            new_cost = sum(distance_matrix[new_solution[i]][new_solution[i + 1]] for i in range(n - 1))
        if new_cost < current_cost or random.random() < np.exp((current_cost - new_cost) / temp):
            current_solution = new_solution
            current_cost = new_cost
            if new_cost < best_cost:
                best_solution = new_solution
                best_cost = new_cost
        temp *= cooling_rate
    return best_cost, best_solution


print(simulated_annealing(distance_matrix_bis))


(2506.6590787153805, [83, 92, 96, 16, 37, 7, 56, 6, 88, 0, 34, 59, 4, 52, 25, 94, 1, 90, 39, 18, 80, 23, 33, 41, 71, 62, 86, 89, 69, 93, 43, 79, 27, 63, 50, 91, 31, 48, 45, 11, 30, 76, 98, 57, 77, 3, 74, 26, 84, 64, 95, 61, 14, 44, 70, 28, 73, 51, 85, 42, 87, 82, 10, 60, 78, 40, 21, 81, 53, 47, 20, 9, 35, 8, 99, 32, 67, 66, 29, 15, 17, 12, 49, 68, 2, 54, 19, 65, 55, 72, 24, 13, 97, 58, 46, 75, 36, 5, 38, 22])


In [57]:

def ant_colony_optimization(distance_matrix, n_ants=20, n_iterations=100, alpha=1, beta=5, evaporation_rate=0.5, circular=True):
    n = len(distance_matrix)
    pheromone = np.ones((n, n))
    visibility = 1 / (np.array(distance_matrix) + 1e-10)
    for _ in range(n_iterations):
        all_tours = []
        for _ in range(n_ants):
            tour = [random.randint(0, n - 1)]
            while len(tour) < n:
                current_city = tour[-1]
                probabilities = [(pheromone[current_city][j] ** alpha) * (visibility[current_city][j] ** beta) if j not in tour else 0 for j in range(n)]
                probabilities = np.array(probabilities) / sum(probabilities)
                next_city = np.random.choice(range(n), p=probabilities)
                tour.append(next_city)
            all_tours.append(tour)
        pheromone *= (1 - evaporation_rate)
        for tour in all_tours:
            if circular:
                cost = sum(distance_matrix[tour[i]][tour[(i + 1) % n]] for i in range(n))
                tour.append(tour[0])
            else:
                cost = sum(distance_matrix[tour[i]][tour[i + 1]] for i in range(n - 1))
            for i in range(n if circular else n - 1):
                pheromone[tour[i]][tour[(i + 1) % n if circular else i + 1]] += 1 / cost
    best_tour = min(all_tours, key=lambda tour: sum(distance_matrix[tour[i]][tour[(i + 1) % n]] for i in range(n)) if circular else sum(distance_matrix[tour[i]][tour[i + 1]] for i in range(n - 1)))
    best_cost = sum(distance_matrix[best_tour[i]][best_tour[(i + 1) % n]] for i in range(n)) if circular else sum(distance_matrix[best_tour[i]][best_tour[i + 1]] for i in range(n - 1))
    return best_cost, best_tour

print(ant_colony_optimization(distance_matrix,circular = True))

(80, [0, 1, 3, 2, 0])


# Optimisation d'une fonction avec beaucoup de minima locaux